In [6]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import glob


In [7]:
def resize_image(img):
    return cv2.resize(img , (4*124 , 4*64))

In [8]:
def extract_hand_segment(image):

    
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the range of skin color in HSV
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Create a mask using the skin color range
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Dilate the mask to remove small holes in the object
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=4)

    # Apply Gaussian blur to the mask to remove noise
    mask = cv2.GaussianBlur(mask, (5, 5), 100)

    # Find the contours of the object in the mask
    _, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour (which should be the hand)
    contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
    largest_contour = max(contour_sizes, key=lambda x: x[0])[1]

    # Create a mask of the hand contour
    hand_mask = np.zeros_like(mask)
    cv2.drawContours(hand_mask, [largest_contour], 0, 255, cv2.FILLED)

    # Apply the hand mask to the original image to extract the hand
    hand_segment = cv2.bitwise_and(image, image, mask=hand_mask)

    # Return the segmented hand image
    return hand_segment





In [9]:
def preProcessing(img):
    # Check if the image is already grayscale
    if len(img.shape) == 2:
        gray = img
    else:
        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    
    # Extract the hand segment from the image
    hand = extract_hand_segment(img)

    # Resize the hand segment to match the input image dimensions
    resized_hand = cv2.resize(hand, (img.shape[1], img.shape[0]))

    # Apply edge detection to the hand segment
    edges = cv2.Canny(resized_hand, 50, 150)

    # Dilate the edges to fill in gaps
    kernel = np.ones((3, 3), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=2)

    # Apply the dilated edges as a mask to the input image
    mask = cv2.cvtColor(dilated_edges, cv2.COLOR_GRAY2BGR)
    masked_img = np.zeros_like(img)
    masked_img[mask > 0] = img[mask > 0]

    resized = resize_image(masked_img)

    return resized


In [10]:
def edgeDetection(img):
    return cv2.Canny(img, 100, 200)

In [11]:
def resizeImage(img):
    return cv2.resize(img , (128*4 , 64*4))

In [12]:
orb = cv2.ORB_create()

In [13]:
# Prepare the training data
X = []
y = []
D = []
L = []
for i in range(0, 6):
    for j in range(1, 6):
        filename = 'images/{}_men ({}).JPG'.format(i, j)
        img = cv2.imread(filename)
        if  img is None:
            continue    

        finalImage = preProcessing(img)
        keypoints, descriptors = orb.detectAndCompute(finalImage, None)
        D.append(descriptors)
        L.append(i*np.ones(len(descriptors)))
        filename = 'images/{}_woman ({}).JPG'.format(i, j)
        img = cv2.imread(filename)
        if  img is None:
            continue    

        finalImage = preProcessing(img)
        keypoints, descriptors = orb.detectAndCompute(finalImage, None)
        D.append(descriptors)
        L.append(i*np.ones(len(descriptors)))
        


In [14]:
X = np.vstack(D)
Y = np.hstack(L)

In [15]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [16]:
# Train the MLP classifier
clf = MLPClassifier(hidden_layer_sizes=(10000,), max_iter=50000, random_state=42)
clf.fit(X_train, Y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10000,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=50000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [17]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.21918861757354355
